# Single Book Emotion Analysis Test

Test the complete pipeline on a single book:
1. Download from Project Gutenberg
2. Detect chapters with improved patterns
3. Run RoBERTa emotion predictions per chapter
4. Visualize emotional trajectory
5. Generate narrative analysis

In [ ]:
# Check if dependencies are installed, if not install them
import sys
import subprocess

def check_and_install_packages():
    """Check for required packages and install if missing."""
    required = {
        'torch': 'torch',
        'transformers': 'transformers',
        'numpy': 'numpy',
        'pandas': 'pandas',
        'matplotlib': 'matplotlib',
        'seaborn': 'seaborn',
        'requests': 'requests'
    }
    
    missing = []
    for module, package in required.items():
        try:
            __import__(module)
        except ImportError:
            missing.append(package)
    
    if missing:
        print(f"Installing missing packages: {', '.join(missing)}")
        print("This may take a few minutes...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + missing)
        print("✓ Installation complete!")
    else:
        print("✓ All required packages are already installed!")

check_and_install_packages()

## Setup - Install Dependencies

Run this cell first to ensure all required packages are installed.

In [ ]:
import sys
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import torch
from transformers import RobertaTokenizer

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src' / 'analysis'))
from emoPredict_roberta import PlutchikEmotionClassifier, load_model, predict_emotions

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Imports successful!")

## Configuration

In [ ]:
# Book configuration
BOOK_ID = 2701  # Moby Dick for testing improved chapter detection
BOOK_TITLE = "Moby Dick"
BOOK_AUTHOR = "Herman Melville"

# Model path (adjust for your local setup)
MODEL_PATH = "/Users/devindyson/Desktop/troglodytelabs/emoWork/emoBERT/models/best_model.pt"

# Processing options
MAX_CHAPTERS = 10  # Limit for testing (set to None for full book)
SEGMENT_METHOD = "chapter"  # "chapter" or "fixed_chunk"
CHUNK_SIZE = 500  # Words per chunk if using fixed_chunk

print(f"Testing: {BOOK_TITLE} (ID: {BOOK_ID})")
print(f"Model: {MODEL_PATH}")
print(f"Max chapters: {MAX_CHAPTERS if MAX_CHAPTERS else 'All'}")

## 1. Download Book

In [ ]:
def download_gutenberg_book(book_id, max_retries=3):
    """Download a book from Project Gutenberg with fallback URLs."""
    urls = [
        f'https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt',
        f'https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt',
        f'https://www.gutenberg.org/ebooks/{book_id}.txt.utf-8'
    ]
    
    for url in urls:
        for attempt in range(max_retries):
            try:
                print(f"  Trying: {url} (attempt {attempt + 1}/{max_retries})")
                response = requests.get(url, timeout=30)
                
                if response.status_code == 200:
                    try:
                        text = response.content.decode('utf-8')
                    except UnicodeDecodeError:
                        text = response.content.decode('latin-1')
                    
                    print(f"  ✓ Downloaded {len(text):,} characters")
                    return text
            except Exception as e:
                print(f"  ✗ Error: {e}")
                continue
    
    return None

print("Downloading book...")
raw_text = download_gutenberg_book(BOOK_ID)

if raw_text:
    print(f"\n✓ Download successful!")
    print(f"Total characters: {len(raw_text):,}")
else:
    print("\n✗ Download failed!")

In [ ]:
# Preview raw text
print("First 2000 characters:")
print("="*80)
print(raw_text[:2000])
print("="*80)

## 2. Strip Gutenberg Boilerplate

In [ ]:
def strip_gutenberg_boilerplate(text):
    """Remove Project Gutenberg header and footer boilerplate."""
    start_patterns = [
        r'\*\*\* START OF TH(IS|E) PROJECT GUTENBERG EBOOK.*?\*\*\*',
        r'\*\*\*START OF THE PROJECT GUTENBERG EBOOK.*?\*\*\*',
    ]
    
    start_pos = 0
    for pattern in start_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start_pos = match.end()
            break
    
    end_patterns = [
        r'\*\*\* END OF TH(IS|E) PROJECT GUTENBERG EBOOK.*?\*\*\*',
        r'\*\*\*END OF THE PROJECT GUTENBERG EBOOK.*?\*\*\*',
    ]
    
    end_pos = len(text)
    for pattern in end_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            end_pos = match.start()
            break
    
    return text[start_pos:end_pos].strip()

clean_text = strip_gutenberg_boilerplate(raw_text)
removed_pct = (len(raw_text) - len(clean_text)) / len(raw_text) * 100

print(f"Removed {removed_pct:.1f}% boilerplate")
print(f"Clean text: {len(clean_text):,} characters")

In [ ]:
# Preview clean text
print("First 2000 characters of clean text:")
print("="*80)
print(clean_text[:2000])
print("="*80)

## 3. Improved Chapter Detection

Enhanced patterns to handle various chapter formats including titles (e.g., "CHAPTER 1. Loomings.").

In [ ]:
def detect_chapters_improved(text, verbose=True):
    """
    Improved chapter detection with support for chapter titles.
    
    Handles formats like:
    - "CHAPTER 1. Loomings."
    - "CHAPTER I"
    - "Chapter 1: The Beginning"
    """
    # Enhanced patterns (order matters - more specific first)
    patterns = [
        # With titles after number/numeral
        (r'^CHAPTER\s+[IVXLCDM]+[\.:)]?\s+.+$', 'CHAPTER + Roman + Title'),
        (r'^CHAPTER\s+\d+[\.:)]?\s+.+$', 'CHAPTER + Arabic + Title'),
        (r'^Chapter\s+[IVXLCDM]+[\.:)]?\s+.+$', 'Chapter + Roman + Title'),
        (r'^Chapter\s+\d+[\.:)]?\s+.+$', 'Chapter + Arabic + Title'),
        
        # Without titles
        (r'^CHAPTER\s+[IVXLCDM]+\.?\s*$', 'CHAPTER + Roman'),
        (r'^CHAPTER\s+\d+\.?\s*$', 'CHAPTER + Arabic'),
        (r'^Chapter\s+[IVXLCDM]+\.?\s*$', 'Chapter + Roman'),
        (r'^Chapter\s+\d+\.?\s*$', 'Chapter + Arabic'),
        
        # Just numerals/numbers (risky, only if nothing else works)
        (r'^[IVXLCDM]+\.?\s*$', 'Roman only'),
        (r'^\d+\.?\s*$', 'Arabic only'),
    ]
    
    lines = text.split('\n')
    
    # Try each pattern
    for pattern, pattern_name in patterns:
        matches = []
        
        for i, line in enumerate(lines):
            line = line.strip()
            if re.match(pattern, line, re.MULTILINE):
                # Calculate character position
                char_pos = sum(len(l) + 1 for l in lines[:i])
                matches.append((line, char_pos, i))
        
        # Use this pattern if we found a reasonable number of chapters
        if 3 <= len(matches) <= 300:  # Expanded upper limit for long books
            if verbose:
                print(f"✓ Pattern matched: {pattern_name}")
                print(f"  Pattern: {pattern}")
                print(f"  Found: {len(matches)} chapters")
                print(f"  Sample chapters:")
                for i, (title, _, _) in enumerate(matches[:5]):
                    print(f"    {i+1}. {title}")
                if len(matches) > 5:
                    print(f"    ... and {len(matches)-5} more")
            
            # Extract chapter texts
            chapters = []
            for i, (title, pos, line_num) in enumerate(matches):
                # Get text until next chapter (or end)
                if i < len(matches) - 1:
                    next_pos = matches[i + 1][1]
                    chapter_text = text[pos:next_pos]
                else:
                    chapter_text = text[pos:]
                
                chapters.append((title, chapter_text, pos))
            
            return chapters, pattern_name
    
    if verbose:
        print("✗ No reliable chapter pattern found")
        print("  Trying to show potential chapter lines...")
        chapter_like = []
        for i, line in enumerate(lines[:1000]):  # Check first 1000 lines
            line = line.strip()
            if 'chapter' in line.lower() or 'CHAPTER' in line:
                chapter_like.append(f"  Line {i}: {line[:80]}")
        if chapter_like:
            print("  Lines containing 'chapter':")
            for cl in chapter_like[:10]:
                print(cl)
    
    return [], None

# Test chapter detection
print("Testing improved chapter detection...\n")
chapters, pattern_used = detect_chapters_improved(clean_text, verbose=True)

In [ ]:
# Analyze chapter statistics
if chapters:
    chapter_stats = []
    for i, (title, text, pos) in enumerate(chapters, 1):
        word_count = len(text.split())
        char_count = len(text)
        chapter_stats.append({
            'chapter': i,
            'title': title,
            'words': word_count,
            'characters': char_count
        })
    
    df_chapters = pd.DataFrame(chapter_stats)
    
    print("\nChapter Statistics:")
    print("="*80)
    print(f"Total chapters: {len(chapters)}")
    print(f"Average words per chapter: {df_chapters['words'].mean():.0f} (±{df_chapters['words'].std():.0f})")
    print(f"Min words: {df_chapters['words'].min()}")
    print(f"Max words: {df_chapters['words'].max()}")
    print(f"Median words: {df_chapters['words'].median():.0f}")
    
    # Show first 10 chapters
    print("\nFirst 10 chapters:")
    print(df_chapters.head(10).to_string(index=False))
    
    # Visualize distribution
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(df_chapters['words'], bins=30, alpha=0.7, edgecolor='black')
    plt.axvline(df_chapters['words'].mean(), color='r', linestyle='--', label='Mean')
    plt.axvline(df_chapters['words'].median(), color='g', linestyle='--', label='Median')
    plt.xlabel('Words per Chapter')
    plt.ylabel('Frequency')
    plt.title('Chapter Length Distribution')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(df_chapters['chapter'], df_chapters['words'], 'o-', alpha=0.7)
    plt.xlabel('Chapter Number')
    plt.ylabel('Words')
    plt.title('Chapter Length Progression')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("\n✗ No chapters detected - will need to use fixed-chunk segmentation")

## 4. Load RoBERTa Emotion Model

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model
print(f"\nLoading model from {MODEL_PATH}...")
if Path(MODEL_PATH).exists():
    model, thresholds = load_model(MODEL_PATH, device)
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    print("\n✓ Model loaded successfully!")
else:
    print(f"\n✗ Model not found at {MODEL_PATH}")
    print("Please update MODEL_PATH in the configuration cell")
    model = None

## 5. Run Emotion Predictions Per Chapter

In [ ]:
if model and chapters:
    # Limit chapters for testing
    chapters_to_process = chapters[:MAX_CHAPTERS] if MAX_CHAPTERS else chapters
    
    print(f"Processing {len(chapters_to_process)} chapters...\n")
    
    emotional_trajectory = []
    
    for i, (title, text, pos) in enumerate(chapters_to_process, 1):
        # Get word count
        word_count = len(text.split())
        
        # Predict emotions
        results = predict_emotions(text, model, tokenizer, thresholds, device)
        
        # Format emotion scores
        emotions = {r["emotion"]: r["probability"] for r in results}
        
        # Find dominant emotion
        dominant = max(results, key=lambda x: x["probability"])
        
        # Add to trajectory
        emotional_trajectory.append({
            "chapter": i,
            "title": title,
            "word_count": word_count,
            "emotions": emotions,
            "dominant_emotion": dominant["emotion"],
            "intensity": dominant["probability"]
        })
        
        # Progress
        if i % 5 == 0 or i == len(chapters_to_process):
            print(f"  Processed {i}/{len(chapters_to_process)} chapters...")
    
    print("\n✓ Emotion predictions complete!")
else:
    print("Cannot run predictions - model not loaded or no chapters detected")
    emotional_trajectory = []

## 6. Per-Chapter Breakdown

In [ ]:
if emotional_trajectory:
    # Create detailed breakdown
    print("\n" + "="*100)
    print("PER-CHAPTER EMOTIONAL BREAKDOWN")
    print("="*100)
    
    for seg in emotional_trajectory:
        print(f"\nChapter {seg['chapter']}: {seg['title']}")
        print("-" * 100)
        print(f"  Words: {seg['word_count']:,}")
        print(f"  Dominant Emotion: {seg['dominant_emotion']} (intensity: {seg['intensity']:.2%})")
        print(f"  All Emotions:")
        
        # Sort emotions by score
        sorted_emotions = sorted(seg['emotions'].items(), key=lambda x: -x[1])
        for emotion, score in sorted_emotions:
            bar = '█' * int(score * 50)  # Visual bar
            marker = '★' if emotion == seg['dominant_emotion'] else ' '
            print(f"    {marker} {emotion:15s}: {score:5.1%} {bar}")
else:
    print("No emotional trajectory data available")

## 7. Emotional Trajectory Visualization

In [ ]:
if emotional_trajectory:
    # Prepare data for visualization
    chapters_num = [seg['chapter'] for seg in emotional_trajectory]
    
    # Extract emotion scores
    emotion_names = list(emotional_trajectory[0]['emotions'].keys())
    emotion_data = {emotion: [] for emotion in emotion_names}
    
    for seg in emotional_trajectory:
        for emotion in emotion_names:
            emotion_data[emotion].append(seg['emotions'][emotion])
    
    # Plot 1: All emotions over chapters
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. All emotions line plot
    ax = axes[0, 0]
    for emotion in emotion_names:
        ax.plot(chapters_num, emotion_data[emotion], marker='o', label=emotion, alpha=0.7)
    ax.set_xlabel('Chapter')
    ax.set_ylabel('Emotion Intensity')
    ax.set_title('Emotional Trajectory - All Emotions')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # 2. Dominant emotion intensity
    ax = axes[0, 1]
    intensities = [seg['intensity'] for seg in emotional_trajectory]
    dominant_emotions = [seg['dominant_emotion'] for seg in emotional_trajectory]
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(set(dominant_emotions))))
    emotion_colors = dict(zip(set(dominant_emotions), colors))
    
    for i, (chapter, intensity, emotion) in enumerate(zip(chapters_num, intensities, dominant_emotions)):
        ax.bar(chapter, intensity, color=emotion_colors[emotion], alpha=0.7)
    
    # Create legend
    handles = [plt.Rectangle((0,0),1,1, color=emotion_colors[e]) for e in sorted(set(dominant_emotions))]
    ax.legend(handles, sorted(set(dominant_emotions)), bbox_to_anchor=(1.05, 1), loc='upper left')
    
    ax.set_xlabel('Chapter')
    ax.set_ylabel('Intensity')
    ax.set_title('Dominant Emotion Intensity per Chapter')
    ax.grid(True, alpha=0.3, axis='y')
    
    # 3. Emotion distribution
    ax = axes[1, 0]
    emotion_counts = {}
    for seg in emotional_trajectory:
        emotion = seg['dominant_emotion']
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
    
    emotions_sorted = sorted(emotion_counts.items(), key=lambda x: -x[1])
    ax.bar([e[0] for e in emotions_sorted], [e[1] for e in emotions_sorted], alpha=0.7)
    ax.set_xlabel('Emotion')
    ax.set_ylabel('Number of Chapters')
    ax.set_title('Dominant Emotion Distribution')
    ax.tick_params(axis='x', rotation=45)
    
    # 4. Heatmap of all emotions
    ax = axes[1, 1]
    emotion_matrix = np.array([emotion_data[e] for e in emotion_names]).T
    im = ax.imshow(emotion_matrix, aspect='auto', cmap='YlOrRd', interpolation='nearest')
    ax.set_xlabel('Emotion')
    ax.set_ylabel('Chapter')
    ax.set_title('Emotion Heatmap')
    ax.set_xticks(range(len(emotion_names)))
    ax.set_xticklabels(emotion_names, rotation=45, ha='right')
    ax.set_yticks(range(len(chapters_num)))
    ax.set_yticklabels(chapters_num)
    plt.colorbar(im, ax=ax, label='Intensity')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize")

## 8. Narrative Analysis

In [ ]:
if emotional_trajectory:
    # Calculate narrative features
    intensities = [seg['intensity'] for seg in emotional_trajectory]
    climax_idx = np.argmax(intensities)
    climax_chapter = emotional_trajectory[climax_idx]
    
    # Emotional range (variance across emotions)
    all_emotion_scores = {}
    for seg in emotional_trajectory:
        for emotion, score in seg['emotions'].items():
            if emotion not in all_emotion_scores:
                all_emotion_scores[emotion] = []
            all_emotion_scores[emotion].append(score)
    
    emotional_range = np.mean([np.var(scores) for scores in all_emotion_scores.values()])
    
    # Pacing (rate of dominant emotion change)
    emotion_changes = sum(
        1 for i in range(1, len(emotional_trajectory))
        if emotional_trajectory[i]['dominant_emotion'] != emotional_trajectory[i-1]['dominant_emotion']
    )
    pacing_score = emotion_changes / len(emotional_trajectory)
    
    # Average intensity
    avg_intensity = np.mean(intensities)
    
    print("\n" + "="*80)
    print("NARRATIVE ANALYSIS")
    print("="*80)
    print(f"\nBook: {BOOK_TITLE} by {BOOK_AUTHOR}")
    print(f"Chapters analyzed: {len(emotional_trajectory)}")
    print(f"Pattern used: {pattern_used}")
    
    print(f"\n📍 CLIMAX:")
    print(f"  Chapter {climax_chapter['chapter']}: {climax_chapter['title']}")
    print(f"  Emotion: {climax_chapter['dominant_emotion']}")
    print(f"  Intensity: {climax_chapter['intensity']:.2%}")
    
    print(f"\n📊 EMOTIONAL CHARACTERISTICS:")
    print(f"  Average intensity: {avg_intensity:.2%}")
    print(f"  Emotional range (variance): {emotional_range:.4f}")
    print(f"  Pacing score (emotion changes): {pacing_score:.2%}")
    
    print(f"\n🎭 DOMINANT EMOTION BREAKDOWN:")
    for emotion, count in sorted(emotion_counts.items(), key=lambda x: -x[1]):
        pct = count / len(emotional_trajectory) * 100
        print(f"  {emotion:15s}: {count:3d} chapters ({pct:5.1f}%)")
    
    print("\n" + "="*80)
else:
    print("No narrative analysis available")

## 9. Save Results

In [ ]:
if emotional_trajectory:
    # Prepare output
    output = {
        "book_metadata": {
            "book_id": f"gutenberg:{BOOK_ID}",
            "title": BOOK_TITLE,
            "author": BOOK_AUTHOR,
            "total_chapters": len(chapters),
            "processed_chapters": len(emotional_trajectory),
            "pattern_used": pattern_used
        },
        "emotional_trajectory": emotional_trajectory,
        "narrative_features": {
            "climax_chapter": climax_chapter['chapter'],
            "climax_title": climax_chapter['title'],
            "climax_emotion": climax_chapter['dominant_emotion'],
            "climax_intensity": climax_chapter['intensity'],
            "average_intensity": float(avg_intensity),
            "emotional_range": float(emotional_range),
            "pacing_score": float(pacing_score)
        }
    }
    
    # Save to file
    output_dir = Path('../output')
    output_dir.mkdir(exist_ok=True)
    
    filename = f"{BOOK_TITLE.lower().replace(' ', '_')}_analysis.json"
    output_path = output_dir / filename
    
    with open(output_path, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"✓ Results saved to: {output_path}")
    print(f"\nFile size: {output_path.stat().st_size:,} bytes")
else:
    print("No results to save")

## 10. Test Summary

In [ ]:
print("\n" + "="*80)
print("TEST SUMMARY")
print("="*80)

print(f"\n✓ Book downloaded: {BOOK_TITLE}")
print(f"✓ Chapter detection: {len(chapters) if chapters else 0} chapters found")
if pattern_used:
    print(f"  Pattern: {pattern_used}")
print(f"✓ Emotion predictions: {len(emotional_trajectory)} chapters processed")
print(f"✓ Visualizations: Generated")
if emotional_trajectory:
    print(f"✓ Results saved: output/{filename}")

print("\n" + "="*80)
print("✅ Test complete!")
print("="*80)